# Test

In [ ]:
using Yao, QAOA, AdaptiveQuantumAnnealing, Random, Combinatorics, Arpack, Distributions
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

PATH = "/home/ubuntu/Archives/";

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

## SK Model

### Small/Large-gap data

In [ ]:
N = 9

# final time for mean-field
T_final = 10000.

# number of points to get Lyapunov exponent for
npts = 256

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

In [ ]:
subdir = "small_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)

eigvals_small_gap = Dict()
lyapunov_exponents_small_gap = Dict()

for instance_name in instance_names
    # println(instance_name)
    pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"
    seed = match(pattern, instance_name)[1]    

    # couplings = h5read(folder_name * instance_name, "J", J_mat)
    eigvals_small_gap[seed] = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    lyapunov_exponents_small_gap[seed] = h5read(folder_name * instance_name, 
                                                @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
end


In [ ]:
subdir = "large_gaps"
# subdir = "extra_data"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)

eigvals_large_gap = Dict()
lyapunov_exponents_large_gap = Dict()

for instance_name in instance_names
    pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"
    seed = match(pattern, instance_name)[1]    

    # couplings = h5read(folder_name * instance_name, "J", J_mat)
    λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    eigvals_large_gap[seed] = λ
    lyapunov_exponents_large_gap[seed] = h5read(folder_name * instance_name, 
                                                @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))    
end


In [ ]:
avg_eigvals_small = [v[2, :] .- v[1, :] for (k, v) in eigvals_small_gap] |> mean
avg_eigvals_large = [v[2, :] .- v[1, :] for (k, v) in eigvals_large_gap] |> mean;

avg_lyapunov_exponents_small = [v for (k, v) in lyapunov_exponents_small_gap] |> mean
avg_lyapunov_exponents_large = [v for (k, v) in lyapunov_exponents_large_gap] |> mean;

In [ ]:
figure(figsize=(7, 3))
subplot(121)
plot(range(0, 1, 33), avg_eigvals_small, "-o", ms=3)
plot(range(0, 1, 33), avg_eigvals_large, "-s", ms=3)
xlim(0, 1)
ylim(0, 2)
xlabel("\$s\$")
ylabel("Average Exact Gap")

subplot(122)
plot(range(0, 1, npts), avg_lyapunov_exponents_small, "-", ms=3)
plot(range(0, 1, npts), avg_lyapunov_exponents_large, "-", ms=3)
xlim(0, 1)
ylim(0, 0.5)
xlabel("\$s\$")
ylabel("Average \$\\lambda\$")
tight_layout()

## Testing

In [ ]:
# # eigvals_small_gap = Dict()
# eigvals_large_gap = Dict()
# minigaps = Dict()
# for instance_name in instance_names
#     pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"
#     seed = match(pattern, instance_name)[1]    

#     # couplings = h5read(folder_name * instance_name, "J", J_mat)
#     λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
#     minigap = minimum(λ[2, :] .- λ[1, :])
#     minigaps[seed] = minigap
    
#     if minigap < 0.01
#         eigvals_small_gap[seed] = λ
#     else
#         eigvals_large_gap[seed] = λ
#     end
# end


In [ ]:
sorted_minigaps = sort(collect(minigaps), by=x->x[2])
length(sorted_minigaps)

In [ ]:
small_minigaps = sorted_minigaps[1:2000]
large_minigaps = sorted_minigaps[2001:end];

In [ ]:
# mv_list = [k for (k, v) in eigvals_small_gap]
# mv_list = [k for (k, v) in eigvals_large_gap]
# for seed in mv_list
#     try
#         mv(PATH * @sprintf("plots/SK_model/N_%i/", N) * @sprintf("random_SK_instance_N_9_seed_%s.pdf", seed), PATH * @sprintf("plots/SK_model/N_%i/", N) * "large_gaps/" * @sprintf("random_SK_instance_N_9_seed_%s.pdf", seed))
#     catch
#         print(seed, ", ")
#     end
# end

In [ ]:
minigap_locs = []
for (k, v) in eigvals_small_gap
    minigap = minimum(v[2, :] .- v[1, :])
    push!(minigap_locs, findfirst(x -> x == minigap, v[2, :] .- v[1, :]))
end

In [ ]:
findall(x -> x != 30, minigap_locs)